# Normalisation des données

Afin de permettre le traitement des données sur le cloud Amazon, il nous faut normaliser nos données.

In [ ]:
# Installation des dépendances
%pip install boto3
%pip install smart_open
%pip install python-dotenv
# https://github.com/GeospatialPython/pyshp
%pip install pyshp

## Connexion au bucket S3 Amazon

Pour pouvoir travailler avec nos données, nous utilisons un bucket S3 contenant les dataset. Afin de permettre la connexion au service S3, il est nécessaire d'avoir défini un fichier `.env` contenant les variables d'environnements suivantes :

```
AWS_ACCESS_KEY_ID=xxxxxx
AWS_SECRET_ACCESS_KEY=xxxxxx
```

Ensuite, il suffit d'ouvrir une session à l'aide de la librairie `boto3` :

In [2]:
import os, boto3, dotenv
from smart_open import open

dotenv.load_dotenv('./.env', override=True)

session = boto3.Session(
    aws_access_key_id = os.environ['AWS_ACCESS_KEY_ID'],
    aws_secret_access_key = os.environ['AWS_SECRET_ACCESS_KEY'],
)

url = 's3://poc-automatic-land-parcel-recognition/test.txt'

with open(url, 'rb', transport_params = {'client': session.client('s3')}) as reader:
    for line in reader:
        print(line.decode('utf8'))


Content of test.txt file: HELLO WORLD !


## Fichiers Shapefile

L'étape suivante est d'apprendre à traiter les données parcellaires à notre disposition contenues dans un Shapefile.

Le shapefile, ou « fichier de formes » est un format de fichier pour les systèmes d'informations géographiques (SIG). Initialement développé par ESRI pour ses logiciels commerciaux, ce format est désormais devenu un standard de facto, dont les spécifications sont ouvertes. Nous allons donc procéder à la lecture d'un exemple de fichier SHP contenant le "dallage" global de notre zone à couvrir.

Pour cela, nous utilisons la librairie python `pyshp``:

In [3]:
import shapefile

url = 's3://poc-automatic-land-parcel-recognition/dataset-reference-fr/092/'

myshp = open(url + 'dalles.shp', 'rb', transport_params = {'client': session.client('s3')}) # Shapes
mydbf = open(url + 'dalles.dbf', 'rb', transport_params = {'client': session.client('s3')}) # Records
myprj = open(url + 'dalles.prj', 'rb', transport_params = {'client': session.client('s3')}) # Projection
sr = shapefile.Reader(shp=myshp, dbf=mydbf, prj=myprj)

print(sr.bbox) # Bounding Box

for n in range(0, sr.numRecords):
    print(sr.record(n))
    print(sr.shape(n).points)
    
myshp.close()
mydbf.close()
myprj.close()


[635000.0, 6845000.0, 655000.0, 6875000.0]
Record #0: ['./92-2021-0645-6855-LA93-0M20-E080.jp2']
[(645000.0, 6855000.0), (650000.0, 6855000.0), (650000.0, 6850000.0), (645000.0, 6850000.0), (645000.0, 6855000.0)]
Record #1: ['./92-2021-0640-6870-LA93-0M20-E080.jp2']
[(640000.0, 6870000.0), (645000.0, 6870000.0), (645000.0, 6865000.0), (640000.0, 6865000.0), (640000.0, 6870000.0)]
Record #2: ['./92-2021-0650-6860-LA93-0M20-E080.jp2']
[(650000.0, 6860000.0), (655000.0, 6860000.0), (655000.0, 6855000.0), (650000.0, 6855000.0), (650000.0, 6860000.0)]
Record #3: ['./92-2021-0645-6850-LA93-0M20-E080.jp2']
[(645000.0, 6850000.0), (650000.0, 6850000.0), (650000.0, 6845000.0), (645000.0, 6845000.0), (645000.0, 6850000.0)]
Record #4: ['./92-2021-0645-6860-LA93-0M20-E080.jp2']
[(645000.0, 6860000.0), (650000.0, 6860000.0), (650000.0, 6855000.0), (645000.0, 6855000.0), (645000.0, 6860000.0)]
Record #5: ['./92-2021-0650-6870-LA93-0M20-E080.jp2']
[(650000.0, 6870000.0), (655000.0, 6870000.0), (65500

Notre but maintenant est de normaliser notre dataset. En entrée, nous avons les Shapefiles contenant les géométries des parcelles et des fichiers images JP2000 ou GeoTIFF :

![SampleDataOrthophoto.png](doc/SampleDataOrthophoto.png)